In [1]:
from metadrive.envs.metadrive_env import MetaDriveEnv
from utils.scenario_runner import ScenarioRunner
from pathlib import Path
import numpy as np

In [2]:
def get_dt(fps):
    # Assume default value
    DECISION_REPEAT = 5
    interval = 1 / fps
    dt = interval / DECISION_REPEAT
    return dt


get_dt(30)

0.006666666666666666

In [3]:
SAVE_DIR = Path("/media/olek/2TB_HDD/metadrive-data/playground")

In [4]:
seed = 0
FPS_RANGE = [10, 20, 30, 40, 50, 60]
for fps in FPS_RANGE:
    dt = get_dt(fps)
    ScenarioRunner(SAVE_DIR, seed, dt=dt, traffic_density=0.0).run_scenario(
        record=True,
        repeat=True,
    )

[INFO] self.fps = 10
[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 1
[INFO] Calculating max steps with: 
[INFO] V_min = 2.0, self.decision_repeat = 5, self.dt = 0.02, distance = 672.88, round(max_steps) = 3364
[INFO] Launching the scenario with record = True
[INFO] Use Torch PPO expert.
[INFO] Episode ended! Scenario Index: 0 Reason: arrive_dest.
[INFO] Saving gif to /media/olek/2TB_HDD/metadrive-data/playground/10/0.mp4
[INFO] Initializing the env took 0.33s
[INFO] Running the scenario took 4.14s
[INFO] Closing the env took 0.13s
[INFO] Total scenario execution took 4.60s
[INFO] Saving data took 0.01s
[INFO] Running scenario finished.
[INFO] self.fps = 20
[INFO] Environment: MetaDri